In [1]:
import Pkg;
using SpecialFunctions
using Plots; plotly();
using Ipopt
using JuMP


┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /Users/adamkulaczkowski/.julia/packages/Plots/oZheM/src/backends.jl:372


In [24]:
#Initialize Variables

# m^3/g 
k_gac = 2
# g/m^3 
p_gac = 2.1*10^6 
p_soil = 1.3*10^6 
# m^3 
vol = 2*1.82*1.22 
#m_soil = p_soil*vol*(1-f_gac)
#m_gac = p_gac*vol*f_gac
lim = (600*10^-9)*(1.3*10^6)
Co = 2.8785

#m
x = 2
#m/s
v = 0.00000317
#m/hr
v_h = 0.011412
#m^2/s
D = 0.0001268392
#m^2/hr
D_h = 0.4566

unpolluted = (200*10^-9)*(1.3*10^6) 
global weaklypolluted = (600*10^-9)*(1.3*10^6)
heavilypolluted = (1000*10^-9)*(1.3*10^6)

1.3000000000000005

In [90]:
#time it takes to reach 2m [hr]
2/0.011412
vol

4.4408

In [4]:
#No GAC Function
function conc(Co,x,v,t,D)
    C = (Co)*erfc((x - v*t)/(2*sqrt(D*t)))
    return C
end

conc (generic function with 1 method)

In [5]:
#No GAC Concentration Curve
figure1 = plot();
        plot!(figure1, [conc(1,2,0.011412,t,0.4566) for t ∈ 1:176], xlabel = "time [hr]",ylabel = "Concentration [g/m^3]", ylim = [0,1.5])
plot!(figure1, fg_legend = :transparent, legend = :bottomright,ylim= [0,1.5])
hline!([unpolluted], label = "unpolluted")
hline!([weaklypolluted], label = "weakly polluted")
hline!([heavilypolluted], label = "heavily polluted")

<!DOCTYPE html>
 
 
 Plots.jl

In [6]:
#GAC Function
function gac(Co,x,v,t,D,f_gac)
        k_d = k_gac*f_gac
        p_d = 1.3
        theta = 0.5
        R = 1 + (k_d*p_d)/theta 
        C = (Co)*erfc((x -(v/R)*t)/(2*sqrt((D/R)*t)))
    return C
end

gac (generic function with 1 method)

In [49]:
#GAC Concentration Curves
figure = plot(); 
for f_gac=[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]
		plot!(figure, [gac(1,2,0.011412,t,0.4566,f_gac) for t ∈ 1:176], xlabel = "time [hr]",ylabel = "Concentration [g/m^3]", ylim = [0,1])
	end

plot!(figure, fg_legend = :transparent, legend = :outerbottomright,ylim= [0,1], legendtitle ="f_gac")
hline!([unpolluted], label = "unpolluted")
hline!([weaklypolluted], label = "weakly polluted")
hline!([heavilypolluted], label = "heavily polluted")

<!DOCTYPE html>
 
 
 Plots.jl

In [102]:
# Determine the number of times the GAC needs to be replaced in a year 
function time(f_gac)
    t = 9.5
    C = gac(1,2,0.011412,t,0.4566,f_gac)
    while C < (600*10^-9)*(1.3*10^6)
        t = t+0.1
        C = gac(1,2,0.011412,t,0.4566,f_gac)
    end
    return t
end

time (generic function with 1 method)

In [18]:
global times = zeros(10)
i = 1
for f_gac in 0.1:0.1:1
    times[i] = time(f_gac)
    i = i+1
end

In [19]:
times

10-element Array{Float64,1}:
  54.200000000000514
  72.70000000000016
  91.2999999999991
 109.79999999999805
 128.299999999997
 146.89999999999594
 165.3999999999949
 183.89999999999384
 202.3999999999928
 220.99999999999173

In [88]:
global fgac = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
global cost = zeros(10)
j = 1
for i in 1:10
    global replacements = 8760/times[i]
    # Price of GAC [$/ft^3]
    price = 99.95
    # Price in [$/m^3]
    price = price/0.0283168
    
    # Labor Price $15/hr, 2 people and 3 hours = 90
    
    cost[j] = (price*replacements*vol*fgac[j] + replacements*90)
    j = j + 1
end

In [89]:
cost

10-element Array{Float64,1}:
 267886.6462489326
 388590.2675843811
 459820.029354625
 507400.9554350649
 541260.180307584
 566199.7097355751
 585887.5065710336
 601614.1914820118
 614465.9389339674
 624882.1822032942

In [47]:
# Uncertainty Analysis on Initial Concentration
crand = rand(0.9:0.001:1.2,100000);
Cfail = 0;
for x in 1:100000
    C = gac(1*crand[x],2,0.011412,times[1],0.4566,0.1)
    if C > weaklypolluted
        Cfail = Cfail + 1
    end 
end
percentFail = (Cfail/100000)*100

66.679

In [75]:
# Uncertainty Analysis on Initial Concentration
vrand = rand(0.9:0.001:1.2,100000);
vfail = 0;
for x in 1:100000
    C = gac(1,2,0.011412*vrand[x],times[10],0.4566,1)
    if C > weaklypolluted
        vfail = vfail + 1
    end 
end
percentFail = (vfail/100000)*100

66.682

In [81]:
using Cbc

In [100]:
function treatment(C,x,v,D)
    treatment_solution = Model(Cbc.Optimizer)
    @variable(treatment_solution, 1 >= fgac >= 0.1) 
    @variable(treatment_solution, replacements >= 0) 
    @NLconstraint(treatment_solution,c1, replacements == 8760/time(fgac))
    @NLobjective(treatment_solution, Min, (99.95/0.0283168)*replacements*4.4408*fgac + replacements*90)
    optimize!(treatment_solution)
    return fgac
end

treatment (generic function with 2 methods)

In [103]:
solution = treatment(1,2,0.011412,0.4566)

LoadError: Unrecognized function "time" used in nonlinear expression.